# Exemplo de Notebook do Google Collab

Este notebook é apenas um exemplo para a utilização do Google Collab na cadeira de Introdução ao Desenho de Fármacos (módulo de Design Computacional de Fármacos).

Nestes notebooks usa-se a linguagem de programação Python para explorar vários aspectos do Design Computacional de Fármacos. No entanto, a maior parte do código-base encontra-se numa única célula (após a secção "Bases de Python), nas qual estão definidas as funções a usar nos trabalhos práticos. Todos os procedimentos subsequentes são feitos recorrendo apenas a commandos/funções simples.

## Setup

Esta secção tem as definições de funções necessárias para correr as aplicações de química/design de fármacos.

Estas instruções estão divididas em duas células: a primeira faz a instalação do software necessario na máquina remota da Google (demora uns minutos). A segunda célula contém activa os módulos necessários e contém definições de funções necessárias para o trabalho (em geral deverá ser rápida a executar).

É fundamental correr ambas as células, em sequência, para que o resto dos exempos possa correr.

In [1]:
%%capture

! pip install ase
! pip install py3Dmol
! pip install seaborn



!wget -c https://repo.continuum.io/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.8.3-Linux-x86_64.sh
!time bash ./Miniconda3-py37_4.8.3-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -y -c rdkit rdkit 
!time conda install -y -c conda-forge openbabel
#!time conda install -y gxx_linux-64
#!time conda install -q -y -c conda-forge rdkit
#!time conda install  -y -c conda-forge xtb

#wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
#chmod +x Miniconda3-latest-Linux-x86_64.sh
#bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
#conda install  -y -c conda-forge python
#conda install  -y -c conda-forge rdkit
#conda install  -y -c conda-forge xtb
#pip install --user py3Dmol
#pip install --user seaborn

# install XTB
!wget -c https://github.com/grimme-lab/xtb/releases/download/v6.4.0/xtb-210201.tar.xz
!tar xf xtb-210201.tar.xz
!export PATH=/content/xtb-6.4.0/bin:${PATH}
!cd ..



In [4]:
import sys
site_pkg_path='/usr/local/lib/python3.7/site-packages'
if site_pkg_path not in sys.path:
  sys.path.append(site_pkg_path)
import os

import numpy as np
import time
import ase
from collections import defaultdict
from rdkit import Chem
from rdkit.Chem import rdFMCS
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdDistGeom
from rdkit.Chem import MACCSkeys
IPythonConsole.ipython_3d = True
import py3Dmol
import matplotlib.pyplot as plt
import subprocess
import seaborn as sb

## FUNÇÕES ADMINISTRATIVAS ##

def shell(cmd, shell=True):
  "runs a command in the linux shell, adapted from Jimmy Kromann."
  if shell:
    p = subprocess.Popen(cmd, shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  else:
    p = subprocess.Popen(cmd, shell=True, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  output, err = p.communicate()
  return output

def run_xtb(args):
  return shell(f"/content/xtb-6.4.0/bin/xtb {args}")

def get_snapshot(dirname='/content/work_dir',fn=None):
  "Downloads work_dir as a tbz file for archive."
  from google.colab import files
  import os
  import time
  origin = os.path.abspath(os.curdir)
  now=time.localtime()
  date_str=f"{now[0]-2000:02d}{now[1]:02d}{now[2]:02d}-{now[3]:02d}{now[4]:02d}"
  ofn = fn if fn else f"{date_str}-{os.path.split(dirname)[-1]}.tbz"
  os.chdir(dirname)
  os.chdir('..')
  shell(f"tar cjf {ofn} ./{os.path.split(dirname)[-1]}")
  files.download(ofn)
  os.chdir(origin)

### Classe Molecule pode ser util para visualizar ficheiros xyz
class Molecule():
  def __init__(self,fn=None, name=None):
    self.name='None'
    self.symbols=list()
    self.geo=np.array([])
    if fn:
      self.read_xyz(fn)
    if name:
      self.name=name

  def natoms(self):
      return len(self.symbols)
  
  def show(self): 
    p = py3Dmol.view(width=400,height=400)
    p.addModel(self.write_xyz())
    #p.setStyle({'stick':{}})
    p.setStyle({'stick':{},'sphere':{'scale':0.3}})
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    p.show()

  def read_xyz(self, fn):
    "Reads a XYZ file"
    with open(fn,'r') as f:
      data = f.readlines()
    natoms = int(data[0])
    g = list()
    for i in range(2,2+natoms):
      l = data[i].split()
      self.symbols.append(l[0].capitalize())
      g.append(list(map(float,l[1:4])))
    self.geo=np.array(g)

  def centre_location(self):
    "returns the location of the geometrical centre as a vector"
    return self.geo.mean(axis=0)

  def shift(self, displacement):
    "Displaces the molecule as a whole, given a 3D vector (np.array)"
    self.geo += displacement

  def center(self):
    "Displaces the molecule so that its geometrical centre is at the origin."
    self.shift(-self.centre_location())

  def write_xyz(self,fn=None):
    s = f"{self.natoms()}\n Created by Molecule Class\n"
    for n in range(self.natoms()):
        s += f"{self.symbols[n]:3s}"
        for i in range(3):
          s += f" {self.geo[n,i]:16.6f}"
        s += '\n'
    if fn:
      with open(fn,'w') as f:
        f.write(s)
    else:
      return s




## SMILES, Fingerprints e Semelhança Molecular

Procedimento:
1. Criamos uma lista de moléculas (formato SMILES)
2. Convertemos cada elemento dessa lista num objecto Molecule do Rdkit
3. Calculamos a Fingerprint de cada composto. Vamos usar duas fingerprints diferentes:
  1. MACCS keys
  2. Morgan Fingerprints de raio 2
4. Calculamos a semelhrança entre pares de moléculas usando a função `DataStructs.FingerprintSimilarity()`

Pelo meio, vamos explorar um pouco melhor a estrutura das fingerprints.



In [7]:
#1
mol_lst = ['CC(=O)C','CCCOC(O)C','C(O)C(O)C(O)','CC(CCO)COC(OCC)C','C=CCC(O)CC=CC=O']


#2
mols=list()

for mol in mol_lst:
  my_mol=Chem.MolFromSmiles(mol)
  my_mol=Chem.AddHs(my_mol)
  # não precisamos de fazer a estrutura 3D, mas podemos correr a linha seguinte
  #AllChem.EmbedMolecule(my_mol)
  mols.append(my_mol)

#3.1

macss_fps = [Chem.MACCSkeys.GenMACCSKeys(x) for x in mols]

#3.2
morgan_fps = [AllChem.GetMorganFingerprint(x,2) for x in mols]



### Notas

```
>>> from rdkit.Chem import Draw
>>> mol = Chem.MolFromSmiles('c1ccccc1CC1CC1')
>>> bi = {}
>>> fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, bitInfo=bi)
>>> bi[872]
((6, 2),)
>>> mfp2_svg = Draw.DrawMorganBit(mol, 872, bi, useSVG=True)
>>> rdkbi = {}
>>> rdkfp = Chem.RDKFingerprint(mol, maxPath=5, bitInfo=rdkbi)
>>> rdkbi[1553]
[[0, 1, 9, 5, 4], [2, 3, 4, 9, 5]]
>>> rdk_svg = Draw.DrawRDKitBit(mol, 1553, rdkbi, useSVG=True)

# This is formatted as code
```

